# Demo del Sistema Multi-Agente para Venta de Coches

¡Bienvenido a la demostración de nuestro sistema avanzado de agentes de IA para la venta de coches!

En este notebook, exploraremos los componentes clave que hacen posible esta solución. Veremos cómo hemos estructurado nuestro sistema multi-agente, cómo cada agente está diseñado para cumplir un rol específico y cómo interactúan entre sí para crear una experiencia de venta fluida e inteligente.

**Objetivos de la demo:**
- **Presentar la arquitectura del sistema:** Conoceremos a nuestros agentes principales: Carlos (el vendedor), María (la investigadora) y el Manager (el coordinador).
- **Analizar la ingeniería de prompts:** Descubriremos cómo hemos moldeado la personalidad y el comportamiento de nuestros agentes a través de prompts cuidadosamente diseñados.
- **Explorar la lógica de negocio:** Veremos cómo el sistema maneja tareas complejas como búsquedas inteligentes en el inventario, negociaciones de precios y la recuperación de información específica de los vehículos.

¡Empecemos!


## Creando la Interfaz de Usuario con Streamlit

Una de las ventajas de este sistema es que podemos crear una interfaz de usuario interactiva de manera muy sencilla utilizando **Streamlit**. En lugar de complejas tecnologías de frontend, Streamlit nos permite construir aplicaciones web directamente en Python.

La interfaz que hemos utilizado para esta demo se encuentra en el fichero `enhanced_app.py`. Con unas pocas líneas de código, podemos crear:
- Un área de chat para interactuar con Carlos.
- Paneles laterales para configurar el sistema.
- Visualizaciones en tiempo real del perfil del cliente y del estado del sistema.

Este enfoque nos permite centrarnos en la lógica de los agentes de IA, mientras que la interfaz se construye de forma rápida y eficiente. A continuación, se muestra un fragmento de código que ilustra lo fácil que es configurar la estructura general de la página, incluyendo la barra lateral y el área de chat principal.


In [ ]:
# Fragmento de: enhanced_app.py

# --- Configuración de la Página ---
st.set_page_config(
    page_title="CarBot Pro - AI Car Salesman", 
    page_icon="🚗",
    layout="wide"
)

# --- Configuración de la Barra Lateral ---
with st.sidebar:
    st.header("🔧 Configuración del Sistema")
    # ... aquí irían los inputs para las API keys y el botón de inicialización ...
    st.subheader("📊 Estado del Sistema")
    if st.session_state.get('system_initialized', False):
        st.markdown('<div>🟢 Sistema Operativo</div>', unsafe_allow_html=True)
    else:
        st.warning("⚠️ Sistema no inicializado")


# --- Estructura Principal con Columnas ---
col1, col2 = st.columns([2, 1]) # Se crea una columna principal más ancha y una secundaria más estrecha

with col1: # Columna para el Chat
    st.subheader("💬 Chat con CarBot Pro")

    # Inicializar historial de mensajes si no existe
    if "messages" not in st.session_state:
        st.session_state.messages = []
        # ... mensaje de bienvenida ...

    # Mostrar mensajes existentes
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # Capturar la entrada del usuario
    if user_input := st.chat_input("¿Qué estás buscando hoy?"):
        # ... lógica del chat ...
        st.rerun()

with col2: # Columna para la Información del Sistema
    st.subheader("⚙️ Información del Sistema y Cliente")
    
    with st.expander("👤 Perfil del Cliente (detectado por Carlos)", expanded=True):
        # ... aquí se muestra la información del perfil del cliente ...
        st.info("El perfil se actualiza en tiempo real.")



## Arquitectura del Sistema: Un Equipo de Agentes Especializados

Nuestro sistema se basa en una arquitectura multi-agente, donde cada agente tiene un rol y un conjunto de habilidades bien definidos. Esta especialización nos permite manejar la complejidad del proceso de venta de manera más eficiente y robusta.

### Los Agentes

- **Carlos (El Vendedor Experto)**: Es el agente principal que interactúa con el cliente. Su personalidad está diseñada para ser carismática, empática y consultiva. Utiliza el framework `ReAct` (Reasoning and Acting) para tomar decisiones y utilizar herramientas.

- **María (La Investigadora de Mercado)**: Es la especialista en buscar información externa. Cuando Carlos necesita datos que no están en nuestro inventario (como reseñas de coches, comparativas de mercado o especificaciones de seguridad detalladas), se lo encarga a María.

- **El Manager (El Coordinador de Negocio)**: Es un agente basado en reglas que actúa como el centro de la lógica de negocio. Gestiona el acceso al inventario, define las estrategias de precios y descuentos, y establece las prioridades de venta.

A continuación, veremos el corazón del sistema: la clase `AdvancedCarSalesSystem` del fichero `advanced_multi_agent_system.py`, donde estos agentes son orquestados.


In [ ]:
# Fragmento de: advanced_multi_agent_system.py

class AdvancedCarSalesSystem:
    """Advanced multi-agent car sales system with professional workflows"""
    
    def __init__(self, openai_api_key: str, serpapi_api_key: str = None):
        self.openai_api_key = openai_api_key
        self.serpapi_api_key = serpapi_api_key
        
        # Initialize inventory manager
        self.inventory_manager = get_inventory_manager()
        
        # Initialize customer profile
        self.customer_profile = CustomerProfile()
        self.sales_stage = SalesStage.GREETING
        
        # Communication system
        self.agent_communications = []
        self.conversation_log = []
        self.carlos_customer_notes: List[str] = [] # Carlos's customer notes
        
        # Initialize LLMs with latest models
        self.carlos_llm = ChatOpenAI(
            temperature=0.8,
            openai_api_key=openai_api_key,
            model_name="gpt-4o",
            max_tokens=1000
        )
        
        self.maria_llm = ChatOpenAI(
            temperature=1,
            openai_api_key=openai_api_key,
            model_name="o4-mini",
            max_tokens=800
        )
        
        self.manager_llm = ChatOpenAI(
            temperature=0.4,
            openai_api_key=openai_api_key,
            model_name="gpt-4o",
            max_tokens=600
        )
        
        # Initialize memory for each agent
        self.carlos_memory = ConversationBufferWindowMemory(
            memory_key="chat_history",
            input_key="input",
            k=10,
            return_messages=True
        )
        
        # Initialize tools and agents
        self.tools = self._create_advanced_tools()
        self.carlos_agent = self._create_carlos_agent()
        self.maria_agent = self._create_maria_agent()
        self.manager_agent = self._create_manager_agent()
        
        logger.info("🚀 Advanced Car Sales System initialized successfully")


### El Cerebro de Carlos: Ingeniería de Prompts

El comportamiento de Carlos no está programado de forma tradicional, sino que se define a través de un "prompt" cuidadosamente diseñado. Este prompt actúa como su hoja de ruta, dándole una personalidad, un proceso de venta a seguir, y directrices sobre cómo utilizar sus herramientas.

Aquí está el prompt completo. Fíjate en cómo definimos su `PERSONALIDAD Y ESTILO`, el `PROCESO DE VENTA ESTRUCTURADO`, y las `DIRECTIVAS CLAVE` para el uso de herramientas. Esta es la base de su capacidad para razonar y actuar.


In [ ]:
# Fragmento de: advanced_multi_agent_system.py

carlos_prompt = PromptTemplate.from_template("""
Eres Carlos, un vendedor de coches experto con 15 años de experiencia, potenciado por IA avanzada (GPT-4o). 
Tu MISIÓN es guiar al cliente a través del proceso de venta para encontrar su coche ideal y cerrar la venta.
Debes ser carismático, conocedor y genuinamente preocupado por las necesidades del cliente.

PERSONALIDAD Y ESTILO:
- Cálido, profesional y confiable.
- Escuchas activamente y haces preguntas inteligentes para descubrir necesidades.
- Usas técnicas de venta consultiva. Nunca seas pasivo, siempre guía la conversación.
- Construyes rapport genuino.
- Manejas objeciones con empatía y datos concretos.

PROCESO DE VENTA ESTRUCTURADO (usa la herramienta UpdateSalesStage para transicionar):
1. GREETING: Saludo inicial, construir rapport.
2. DISCOVERY: Entender profundamente necesidades, presupuesto, preferencias del cliente.
3. PRESENTATION: Mostrar vehículos del inventario que coincidan perfectamente (usar ConsultManager para obtener opciones de inventario).
4. OBJECTION_HANDLING: Abordar preocupaciones con empatía y soluciones.
5. NEGOTIATION: Trabajar hacia un acuerdo (usar ConsultManager para precios/descuentos y consultas de inventario complejas).
6. CLOSING: Finalizar la venta de manera natural.

HERRAMIENTAS DISPONIBLES (DEBES usar estas herramientas para interactuar con el sistema):
{tools}

USO DE HERRAMIENTAS - DIRECTIVAS CLAVE:
- `ConsultManager`: ES TU HERRAMIENTA PRINCIPAL. Úsala para TODO lo relacionado con el inventario de la concesionaria (búsquedas, disponibilidad, VINs, detalles específicos de un coche que tenemos), precios, descuentos, y directivas de venta. NO uses ResearchVehicleInfo para esto.
- `ResearchVehicleInfo`: Úsala SOLO para obtener información externa del mercado que NO está en nuestro inventario, como reseñas de un modelo en general, comparativas con competidores, o datos de seguridad de una fuente externa.
- `FinalizeSaleAndReserveVehicle`: Úsala ÚNICAMENTE para cerrar la venta cuando el cliente confirme la compra. NECESITAS el VIN exacto, que debes obtener con `ConsultManager`.

LISTA DE NOMBRES DE HERRAMIENTAS: {tool_names}

RAZONAMIENTO FLEXIBLE:
- Si el cliente cambia de parecer (ej. pregunta por "BMWs" después de haber hablado de sedanes rojos), DEBES relajar tus criterios de búsqueda. Haz una consulta más amplia al manager (ej. "Busca BMWs seguros para familia") en lugar de mantener todos los filtros anteriores.
- Si una herramienta te da la información que necesitas (ej. `ResearchVehicleInfo` te da los datos de seguridad), ¡úsala! Sintetiza la información y responde al cliente. No vuelvas a preguntar al manager lo mismo.

CONTEXTO ACTUAL:
Etapa de venta actual: {sales_stage}
Perfil del cliente (actualizado continuamente): {customer_profile_summary}
Comunicaciones internas recientes (Manager/Maria): {internal_communications_summary}
TUS NOTAS PERSONALES DEL CLIENTE (Usa UpdateCustomerNotes para gestionarlas):
{customer_notes_summary}

INSTRUCCIONES CRÍTICAS PARA RESPONDER (DEBES SEGUIR ESTE FORMATO):
...
""")


### Las Herramientas de Carlos: Extendiendo sus Capacidades

Para que Carlos sea más que un simple chatbot, lo hemos equipado con un conjunto de "herramientas" que puede decidir usar de forma autónoma. Estas herramientas le dan acceso a funcionalidades clave del sistema, permitiéndole realizar acciones concretas como consultar el inventario, actualizar el perfil del cliente o finalizar una venta.

Las herramientas más interesantes son aquellas que le permiten gestionar el estado de la conversación. Sin ellas, Carlos sería un agente sin memoria, incapaz de seguir un proceso de venta coherente.

- **`UpdateCustomerProfile`**: Esta herramienta permite a Carlos construir un perfil dinámico del cliente a lo largo de la conversación. Almacena datos clave como el presupuesto, las preferencias de color o las necesidades específicas (como la seguridad para un bebé). Esto le permite personalizar sus recomendaciones y no tener que preguntar lo mismo varias veces.

- **`UpdateSalesStage`**: El proceso de venta tiene varias etapas (descubrimiento, presentación, negociación, etc.). Esta herramienta le permite a Carlos ser consciente de en qué punto del proceso se encuentra y ajustar su comportamiento y lenguaje en consecuencia. Por ejemplo, no intentará cerrar una venta en la etapa de descubrimiento.

- **`UpdateCustomerNotes`**: A veces, los clientes dan información valiosa pero informal que no encaja en un campo estructurado del perfil. Esta herramienta le da a Carlos una "libreta de notas" personal donde puede apuntar detalles cualitativos, como "al cliente le preocupan los costes de mantenimiento" o "mencionó que tiene dos perros". Estos apuntes le ayudan a crear una conexión más personal y a recordar detalles importantes más adelante.

A continuación, se muestra el código que crea estas herramientas.


In [ ]:
# Fragmento de: advanced_multi_agent_system.py

def _create_advanced_tools(self) -> List[Tool]:
    """Create advanced tools for the multi-agent system"""
    tools = []
    
    # ... (otras herramientas como ConsultManager y ResearchVehicleInfo)
    
    # Customer profiling tool
    def update_customer_profile(info: str) -> str:
        """Update customer profile with new information"""
        self._log_agent_action(AgentRole.CARLOS_SALES, "profile_update", info)
        
        try:
            self._update_customer_profile_from_text(info)
            profile_summary = self._get_customer_profile_summary()
            
            logger.info(f"📝 Customer profile updated: {profile_summary}")
            return f"Perfil actualizado: {profile_summary}"
            
        except Exception as e:
            logger.error(f"❌ Error updating customer profile: {e}")
            return "Error actualizando el perfil del cliente."
    
    tools.append(Tool(
        name="UpdateCustomerProfile",
        func=update_customer_profile,
        description="Update customer profile with preferences, needs, budget, and other relevant information"
    ))
    
    # Sales stage management
    def update_sales_stage(stage: str) -> str:
        """Update the current sales stage"""
        try:
            new_stage = SalesStage(stage.lower())
            old_stage = self.sales_stage
            self.sales_stage = new_stage
            
            self._log_agent_action(
                AgentRole.CARLOS_SALES,
                "stage_transition",
                f"{old_stage.value} -> {new_stage.value}"
            )
            
            return f"Etapa de venta actualizada a: {new_stage.value}"
            
        except ValueError:
            return f"Etapa de venta inválida: {stage}"
    
    tools.append(Tool(
        name="UpdateSalesStage",
        func=update_sales_stage,
        description="Update the current sales stage (greeting, discovery, presentation, objection_handling, negotiation, closing)"
    ))

    # Carlos's Customer Notes tool
    def update_customer_notes(note_to_add: str, mode: str = "append") -> str:
        """Adds or overwrites notes in Carlos's personal customer notepad."""
        # ... (lógica para añadir o sobrescribir notas)
        if mode.lower() == "overwrite":
            self.carlos_customer_notes = [note_to_add]
        elif mode.lower() == "append":
            self.carlos_customer_notes.append(note_to_add)
        return f"Nota gestionada."

    tools.append(Tool(
        name="UpdateCustomerNotes",
        func=update_customer_notes,
        description="Gestiona tus notas personales sobre el cliente. Útil para detalles cualitativos. Modos: 'append', 'overwrite'."
    ))
    
    return tools


### María: La Especialista en Investigación

Mientras que el Manager se ocupa de la información interna, María es nuestra experta en el mundo exterior. Su propósito es enriquecer la conversación con datos objetivos y fiables que no se encuentran en nuestro inventario.

Cuando un cliente tiene preguntas específicas sobre la seguridad de un modelo, quiere ver comparativas de mercado, o conocer la opinión de expertos sobre un vehículo, Carlos le delega la tarea a María.

El `_maria_research_engine` utiliza `SerpAPI` para realizar búsquedas web en tiempo real. Pero no se limita a devolver una lista de enlaces; utiliza un segundo LLM (`o4-mini`) para analizar los resultados de la búsqueda y redactar un informe conciso y estructurado para Carlos. Esto le ahorra a Carlos el trabajo de tener que leer múltiples fuentes, dándole directamente la información que necesita para responder al cliente.


In [ ]:
# Fragmento de: advanced_multi_agent_system.py

def _maria_research_engine(self, query: str) -> str:
    """Maria's research engine for vehicle information"""
    logger.info(f"🔬 MARIA RESEARCH REQUEST: {query}")
    
    raw_search_snippets = ""
    source_type = ""
    
    # Try SerpAPI first if available
    if self.serpapi_api_key:
        try:
            search_wrapper = SerpAPIWrapper(serpapi_api_key=self.serpapi_api_key)
            raw_search_snippets = search_wrapper.run(f"car review {query} 2023 2024 specifications safety reliability comparisons")
            source_type = "Búsqueda Web (SerpAPI)"
            logger.info("Maria completed web research successfully via SerpAPI.")
        except Exception as e:
            logger.warning(f"⚠️ SerpAPI research failed: {e}. Falling back to knowledge base.")
            raw_search_snippets = self._knowledge_based_research(query, internal_call=True) # Get raw data
        source_type = "Base de Conocimiento Interna"

    # Now, Maria analyzes these snippets
    maria_analyzer_prompt_text = (
        "Eres María, una investigadora de coches experta y analítica. Carlos, un vendedor, te ha hecho la siguiente consulta:\\n"
        "CONSULTA DE CARLOS: \\"{carlos_query}\\"\\n\\n"
        "Has recopilado los siguientes fragmentos de información sin procesar de {source_type}:\\n"
        # ... (el resto del prompt de análisis)
    )
    # ...
    
    try:
        logger.info(f"🧠 Maria (gpt-5-mini) está analizando los fragmentos de: {source_type}")
        analytical_report = self.maria_llm.invoke(analyzer_prompt).content
        logger.info(f"✅ Maria (gpt-5-mini) completó el análisis.")
        
        report_parts = [
            f"🔬 **INFORME DE INVESTIGACIÓN DE MARÍA:**",
            f"\\n**Consulta Original de Carlos:** \\"{query}\\"",
            f"**Fuentes Consultadas:** {source_type}",
            f"\\n{analytical_report}",
            # ... (resto del formato del informe)
        ]
        final_report = "\\n".join(report_parts)
        return final_report.strip()

    except Exception as e:
        logger.error(f"❌ Error durante el análisis de María (gpt-5-mini): {e}")
        return f"🔬 Error en el análisis de María."


### El Manager: La Lógica de Negocio

A diferencia de Carlos, que es un agente flexible basado en un LLM, el Manager es un agente más estructurado y basado en reglas. Su función es aplicar la lógica de negocio de manera consistente.

El `_manager_decision_engine` es el método principal que recibe las consultas de Carlos. Actúa como un enrutador, distribuyendo la solicitud a la función adecuada según su contenido (búsquedas de inventario, solicitudes de VIN, negociaciones de precios, etc.).

A continuación, se muestra el código del motor de decisiones y de la función de búsqueda en el inventario. Fíjate en cómo hemos mejorado la extracción de palabras clave para hacer las búsquedas más inteligentes y relevantes.


In [ ]:
# Fragmento de: advanced_multi_agent_system.py

def _manager_decision_engine(self, request: str) -> str:
    """Manager's decision-making engine for business policies"""
    logger.info(f"🏢 MANAGER CONSULTATION: {request}")
    
    request_lower = request.lower()
    
    # VIN request
    if 'vin' in request_lower:
        return self._handle_vin_request(request)

    # Inventory search request from Carlos
    if any(keyword in request_lower for keyword in ["buscar coche", "opciones de vehículo", "inventario", "búsqueda de coches", "inventory search", "buscar en inventario"]):
        logger.info(f"🏢 Manager received inventory search request: {request}")
        # Extract the actual query part for the inventory search.
        # This is a simple heuristic; a more robust NLP approach might be needed for complex requests.
        search_query = request # Default to full request
        
        # Improved keyword extraction
        keywords = []
        # Extract brand names mentioned
        brands = ['audi', 'bmw', 'mercedes', 'toyota', 'honda', 'ford', 'volkswagen', 'alfa romeo']
        for brand in brands:
            if brand in request_lower:
                keywords.append(brand)
        
        # Extract other meaningful terms
        other_terms = ['seguro', 'familiar', 'bebé', 'espacioso', 'rojo', 'sedan', 'suv', 'deportivo']
        for term in other_terms:
            if term in request_lower:
                keywords.append(term)
        
        if keywords:
            search_query = ' '.join(keywords)
        else: # Fallback to original, simpler extraction if no keywords found
            if "necesito opciones de" in request_lower:
                 search_query = request[request_lower.find("necesito opciones de") + len("necesito opciones de"):].strip()
            elif "busca un" in request_lower:
                 search_query = request[request_lower.find("busca un") + len("busca un"):].strip()
            elif "buscando" in request_lower:
                 search_query = request[request_lower.find("buscando") + len("buscando"):].strip()
            elif "query:" in request_lower: # If Carlos explicitly passes a query
                 search_query = request[request_lower.find("query:") + len("query:"):].strip()
        
        if not search_query or search_query == request: # Fallback if extraction is not specific enough
             # Try to remove common phrases if they are the whole request
            phrases_to_remove = ["el cliente busca", "necesito opciones del inventario", "realiza una búsqueda de inventario para", "inventory search for"]
            for phrase in phrases_to_remove:
                if phrase in request_lower:
                    search_query = request_lower.replace(phrase, "").strip()
                    break
        
        logger.info(f"🛠️ Manager extracted search query: '{search_query}'")
        if not search_query.strip() or search_query.strip() == ".": # Avoid empty searches
            logger.warning("⚠️ Manager received an empty or too generic search query. Asking for clarification.")
            return "Por favor, especifica mejor qué tipo de vehículos necesita el cliente para la búsqueda en inventario."

        search_results_objects = self.inventory_manager.intelligent_search(search_query, max_results=8) # Assuming this returns list of CarSearchResult
        
        if not search_results_objects:
            return f\"\"\"
🏢 **RESPUESTA DEL MANAGER - BÚSQUEDA DE INVENTARIO:**

No se encontraron vehículos que coincidan con los criterios: '{search_query}'.
Por favor, informa al cliente e intenta con criterios más amplios si es posible.
            \"\"\"
...


### El Inventario Inteligente: `EnhancedInventoryManager`

El `EnhancedInventoryManager` es el componente responsable de interactuar con nuestra base de datos de vehículos (el fichero `enhanced_inventory.csv`). Su función más importante es la búsqueda inteligente.

El método `intelligent_search` no es una simple búsqueda por palabras clave. Intenta comprender la intención del usuario a partir de una consulta en lenguaje natural, extrayendo criterios específicos como el rango de precios, el color, la marca o el tipo de vehículo.

Veamos el código de esta función y cómo descompone una consulta para filtrar y puntuar los resultados.


In [ ]:
# Fragmento de: enhanced_inventory_manager.py

def intelligent_search(self, query: str, max_results: int = 10) -> List[CarSearchResult]:
    """Advanced intelligent search with natural language processing"""
    if self.inventory_df is None or self.inventory_df.empty:
        logger.warning("No inventory data available for search.")
        return []
    
    logger.info(f"🔍 Performing intelligent search: '{query}'")
    
    # Filter out reserved cars by default from the initial DataFrame copy
    active_inventory_df = self.inventory_df[self.inventory_df['status'] == 'Available'].copy()
    if active_inventory_df.empty:
        logger.info("No 'Available' vehicles in inventory to search.")
        return []
    
    # Parse query for specific criteria
    search_criteria = self._parse_search_query(query)
    
    # Apply filters based on criteria to the active inventory
    filtered_df = self._apply_search_filters(active_inventory_df, search_criteria)
    
    # Calculate relevance scores
    scored_results = self._calculate_relevance_scores(filtered_df, query, search_criteria)
    
    # Convert to structured results
    results = self._convert_to_search_results(scored_results, max_results)
    
    # Log search
    self.search_history.append({
        'timestamp': datetime.now(),
        'query': query,
        'results_count': len(results),
        'criteria': search_criteria
    })
    
    logger.info(f"✅ Found {len(results)} matching vehicles")
    return results


In [ ]:
# Fragmento de: enhanced_inventory_manager.py

def _parse_search_query(self, query: str) -> Dict[str, Any]:
    """Parse natural language query into search criteria"""
    query_lower = query.lower()
    criteria = {}
    
    # Extract price range
    price_patterns = [
        r'menos de (\d+)',
        r'bajo (\d+)',
        r'máximo (\d+)',
        r'hasta (\d+)',
        r'entre (\d+) y (\d+)',
        r'(\d+) a (\d+)',
        r'presupuesto de (\d+)'
    ]
    
    for pattern in price_patterns:
        match = re.search(pattern, query_lower)
        if match:
            if len(match.groups()) == 1:
                criteria['max_price'] = int(match.group(1))
            elif len(match.groups()) == 2:
                criteria['min_price'] = int(match.group(1))
                criteria['max_price'] = int(match.group(2))
            break
    
    # Extract makes
    makes = ['audi', 'bmw', 'mercedes', 'toyota', 'honda', 'ford', 'volkswagen', 
            'nissan', 'hyundai', 'kia', 'mazda', 'subaru', 'lexus', 'acura',
            'infiniti', 'volvo', 'cadillac', 'genesis', 'jaguar', 'land rover',
            'porsche', 'tesla', 'aston martin', 'ferrari', 'lamborghini', 'alfa romeo']
    
    for make in makes:
        if make in query_lower:
            criteria['make'] = make.title()
            break
            
    return criteria


## Conclusión

Como hemos visto, la combinación de agentes de IA especializados nos permite construir sistemas complejos y robustos.

**Puntos Clave:**
- **Ingeniería de Prompts:** La personalidad y el comportamiento de los agentes se definen a través de prompts, lo que permite un control detallado y flexible.
- **División de Responsabilidades:** Asignar roles específicos a cada agente (vendedor, investigador, manager) hace que el sistema sea más modular y fácil de mantener.
- **Lógica Híbrida:** Combinamos la flexibilidad de los LLMs (Carlos) con la consistencia de los agentes basados en reglas (Manager) para obtener lo mejor de ambos mundos.

Este enfoque nos permite crear soluciones de IA que no solo son inteligentes, sino también prácticas y alineadas con los objetivos de negocio.

¡Gracias por acompañarnos en esta demo!
